In [1]:
## packages
import collections
import numpy as np
import textblob as tb
from textblob import TextBlob as TB
import math
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from scipy.spatial import distance
stopwords = set(stopwords.words('english'))

import pprint as pp
import pandas as pd

[nltk_data] Downloading package stopwords to /Users/Shawn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Regressors

from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import ElasticNet
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.multioutput import MultiOutputRegressor

from sklearn.cross_decomposition import PLSRegression
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import AdaBoostRegressor

# from sklearn.tree import DecisionTreeRegressor

In [3]:
## random split to 8000:2000 (train:validate)
# train_indice = np.random.choice(10000, size=10000, replace=False)
# validate_indice = [item for item in range(10000) if item not in train_indice]

## fix split to 8000:2000 (train:validate)
train_indice = range(10000)
# validate_indice = range(8000,10000)
test_indice = range(2000)

In [4]:
def reorder_pd(label, indice):
    new_label=label
    for i in range(indice):
        img=new_label.iloc[i][0]
        # find t, image #
        t=''
        for j in range(len(img)):
            if img[j]=='/':
                if img[j+4]=='.':
                    t+=img[j+1:j+4]
                elif img[j+3]=='.':
                    t+=img[j+1:j+3]
                elif img[j+5]=='.':
                    t+=img[j+1:j+5]
                elif img[j+2]=='.':
                    t+=img[j+1:j+2]
                else:
                    t+=img[j+1]
                
        new_label.set_value(i,0,int(t))
    return new_label.sort_values(by=[0])

In [5]:
# sort train_fc1000
train_fc1 = pd.DataFrame(pd.read_csv("data/features_train/features_resnet1000_train.csv", header=-1))
train_sorted_fc1=reorder_pd(train_fc1, 10000)
train_sorted_fc1000 = train_sorted_fc1.drop(0, 1).as_matrix()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [6]:
# sort test_fc1000
test_fc1 = pd.DataFrame(pd.read_csv("data/features_test/features_resnet1000_test.csv", header=-1))
test_sorted_fc1=reorder_pd(test_fc1, 2000)
test_sorted_fc1000 = test_sorted_fc1.drop(0, 1).as_matrix()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [7]:
%%time

# training, size of 10000
def generate_word_space():
    total_count = collections.defaultdict(int)
    for i in train_indice:
        path = "data/descriptions_train/" + str(i) + ".txt"
        fo = open(path, "r")
        for line in fo.readlines():
            line = line.strip()
            ret = TB(line).lower()

            sentence_words=[]
            for word,pos in ret.tags:
                if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS') and word not in stopwords:
                    sentence_words.append(tb.blob.Word(word).stem())

            for index in range(len(sentence_words)):
                total_count[sentence_words[index]] += 1
    return total_count

## -> noun : count
def generate_nouns(file_path, indice):
    image_nouns = []
    for i in indice:
#         current_count = collections.defaultdict(int)
        current_nouns = collections.defaultdict(float)
        path = file_path + str(i) + ".txt"
        fo = open(path, "r")
        for line in fo.readlines():
            line = line.strip()
            ret = TB(line).lower()
            
            sentence_nouns=[]
            for word,pos in ret.tags:
                if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS') and word not in stopwords:
                    sentence_nouns.append(tb.blob.Word(word).stem())

            for index in range(len(sentence_nouns)):
#                 current_count[sentence_nouns[index]] += 1
#                 current_nouns[sentence_nouns[index]] = math.log2(current_count[sentence_nouns[index]] + 1)
                current_nouns[sentence_nouns[index]] = 1

#             for word_phrase in ret.noun_phrases:
#                 ret_phrase = TB(word_phrase).lower()
#                 for word,pos in ret_phrase.tags:
#                     if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS'):
#                         temp_noun = tb.blob.Word(word).singularize()
#                         current_count[temp_noun] += 1
#                         current_nouns[temp_noun] = math.log2(current_count[temp_noun] + 1)

        image_nouns.append(dict(current_nouns))
    return image_nouns

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10 µs


In [8]:
train_image_nouns = generate_nouns("data/descriptions_train/",train_indice)
test_image_nouns = generate_nouns("data/descriptions_test/",test_indice)

total_count = generate_word_space()
reduced_word_space = dict((k, v) for k, v in total_count.items() if v>=2)
feature_name_list = reduced_word_space.keys()

In [9]:
len(reduced_word_space)

3642

In [10]:
## 
def generate_input_features(indice, image_nouns):
    image_nouns_features=[]
    for i in range(len(indice)):
        label=[0 for d in range(len(feature_name_list))]
        for index,k in enumerate(feature_name_list):
            if k in image_nouns[i]:
                label[index]=image_nouns[i][k]
        image_nouns_features.append(label)
    return image_nouns_features

In [11]:
train_image_nouns_features=generate_input_features(train_indice, train_image_nouns)
test_image_nouns_features=generate_input_features(test_indice, test_image_nouns)


In [12]:
# tmp here, for train and predict
#                         train_sorted_fc1000
#                         train_image_nouns_features

#                         test_sorted_fc1000
#                         test_image_nouns_features

In [13]:

# np.matrix(train_image_nouns_features)
print(type(train_sorted_fc1000))
# print (np.matrix(train_image_nouns_features).shape)

<class 'numpy.ndarray'>


In [132]:
# train model

# ridge = Ridge()
ridge = Ridge(alpha=450, tol = 0.001)
ridge.fit(train_sorted_fc1000, np.matrix(train_image_nouns_features))
train_prediction = ridge.predict(train_sorted_fc1000)
test_prediction = ridge.predict(test_sorted_fc1000)

In [133]:
def closest_node(node, nodes):
    print(int(len(node)/100))
    closest_indice = np.argsort(distance.cdist(node, nodes))[:,:int(len(node)/100)]#[:,::-1]
    return closest_indice

In [134]:
print(len(train_image_nouns_features))
print(len(train_image_nouns_features[0]))


print(len(train_prediction))
print(len(train_prediction[0]))

10000
3642
10000
3642


In [135]:
print(type(np.matrix(train_image_nouns_features)))
print(type(train_prediction))

<class 'numpy.matrixlib.defmatrix.matrix'>
<class 'numpy.ndarray'>


In [136]:
# training score
# train_match = closest_node(np.matrix(train_image_nouns_features), train_prediction)
# train_match.shape

# train_match_index=[]
# train_score = 0
# for i in range(len(train_indice)):
#     if i in train_match[i]:
#         total=(len(train_indice)/100.0)
#         train_score += (total-list(train_match[i]).index(i))/total
#         train_match_index.append(i)
# train_score/10000

In [137]:
%%time

test_match = closest_node(np.matrix(test_image_nouns_features), test_prediction)

# output to file
result = []
for i in range(2000):
    a = [str(i) + '.txt']
    b = ''
    for j in range(20):
        b += str(test_match[i][j]) + '.jpg '
    a.append(b)
    result.append(a)
    
np.savetxt('new.csv', result, delimiter = ',', fmt='%s', comments='', header='Descritpion_ID,Top_20_Image_IDs')
# list(map(lambda x: validate_indice[x],validate_match[0]))

20
CPU times: user 16.4 s, sys: 91 ms, total: 16.4 s
Wall time: 16.6 s


In [ ]:

# 100 -> 0.38476

# 50  -> 0.38674

# 47  -> 0.38754
# 45  -> 0.38830
# 40  -> 0.38758

# 16  -> 0.3861
# 15  -> 0.38448   0.001
# 15  -> 0.38448   0.0001
    # 14  -> 0.38494
# 13  -> 0.38388
# 12  -> 0.38403
# 11  -> 0.38299
# 10  -> 0.38186
# 9.0 -> 0.38164
# 8.0 -> 0.37962
# 7.0 -> 0.37914
# 6.0 -> 0.37646
# 5.0 -> 0.37355
# 4.0 -> 0.37146
# 3.0 -> 0.37075
# 2.0 -> 0.368
# 0.5 -> 0.3606
# 0.1 -> 0.357
# 0.05 -> 0.35627